## Proyecto 1 

Nicolas Arango - 202220342
Mateo Rincon - 202221402
Amalia Carbonell - 202122079 



1. Entendimiento y preparación de los datos (a nivel de código y análisis).
2. Modelado y evaluación.

### Entendimiento y preparación de los datos (a nivel de código y análisis)

In [1]:
import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

In [2]:
# Carga de Datos
#noticias_df = pd.read_csv("./data/fake_news_test.csv", sep=";", usecols=['ID','Titulo', "Descripcion", "Fecha"])
data=pd.read_csv('./data/fake_news_test.csv', sep=';', encoding = 'utf-8')

In [3]:
data.shape


(1000, 4)

In [4]:
data.dtypes

ID             object
Titulo         object
Descripcion    object
Fecha          object
dtype: object

In [5]:
data.head()



,ID,Titulo,Descripcion,Fecha
0,ID,La mesa del congreso censura un encuentro inte...,"Portavoces de Ciudadanos, PNV, UPN, PSOE, Unid...",30/10/2018
1,ID,La brecha digital que dificulta el acceso de a...,No es la primera vez que los ciudadanos vulner...,15/03/2023
2,ID,PP apremia al EQUO a presentar una propuesta d...,El partido morado reprocha que los socialistas...,01/07/2019
3,ID,De soberano garante de la democracia a rey cor...,"La renuncia de Felipe VI a su herencia, proced...",16/03/2020
4,ID,El Gobierno aprobará este martes detraer los b...,El Ejecutivo también prorrogará la suspensión ...,13/09/2021


### Describir los Datos

Se hace una descripción más detallada de nuestros datos.Es un paso es muy importante para poder determinar problemas de calidad de datos. Además, nos dará información que será útil en la fase de preparación de datos.

In [6]:
data.describe()

,ID,Titulo,Descripcion,Fecha
count,1000,999,1000,1000
unique,1,997,995,755
top,ID,La Junta Electoral multa con 1.000 euros a 'AB...,La vicepresidenta ha asegurado que el Gobierno...,24/06/2019
freq,1000,2,2,5


In [7]:
data.sample(5)

,ID,Titulo,Descripcion,Fecha
261,ID,"Albert Comellas, alertador de corrupción: 'Si ...",Denunció un intento de malversación propuesto ...,22/06/2022
877,ID,El ala socialista del Gobierno avisa sobre la ...,"La ministra portavoz, Isabel Rodríguez, ha des...",31/05/2022
936,ID,Vox quiere prohibir los partidos políticos ind...,Iván Espinosa de los Monteros ha asegurado que...,18/03/2019
733,ID,"Eva Kaili, exvicealcalde de Valencia, a juicio...",La Fiscalía Anticorrupción solicita para Grau ...,18/01/2018
732,ID,Marlaska insiste en hablar con el Govern del o...,"El ministro del Interior, mediante una carta r...",28/08/2018
